# Unzip and Data Structuring



In [ ]:
# Import Essential Library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/Project 3 - Kaggle NLP/feedback-prize-2021.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/kaggle/')

In [ ]:
train_all_files = os.listdir('/content/kaggle/train/')    #list all file name in train files
len(train_all_files)

15594

In [ ]:
test_all_files = os.listdir('/content/kaggle/train/')     #list all file name in test files, for submission

In [ ]:
df = pd.read_csv('/content/kaggle/train.csv')            #open csv file 
df

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...
...,...,...,...,...,...,...,...,...
144288,4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...
144289,4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...
144290,4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838
144291,4C471936CD75,1.618025e+12,4570.0,4922.0,The impact of asking people to help you make a...,Evidence,Evidence 4,839 840 841 842 843 844 845 846 847 848 849 85...


In [ ]:
# save all file in a list all_essays
all_essays = []
for text_file in train_all_files:
  with open('./kaggle/train/'+text_file,'r') as f:
    temp_text = f.read()
  all_essays.append(temp_text)

# Preprocessing A


In [ ]:
def type(x):
  if x=='Lead':
    return 0
  elif x=='Position':
    return 1
  elif x=='Evidence':
    return 2
  elif x=='Claim':
    return 3 
  elif x=='Concluding Statement':
    return 4
  elif x=='Counterclaim':
    return 5
  elif x=='Rebuttal':
    return 6


In [ ]:
df['discourse_type_']=df.discourse_type.apply(type)

In [ ]:
df.discourse_type_.value_counts()

3    50208
2    45702
1    15419
4    13505
0     9305
5     5817
6     4337
Name: discourse_type_, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X=df['discourse_text']
y=df['discourse_type_']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)#,shuffle=False)

In [ ]:
num_classes=7
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
X_train[2],y_train[2]

('Some certain areas in the United States ban phones from class rooms just because of it. ',
 array([0., 0., 1., 0., 0., 0., 0.], dtype=float32))

In [ ]:
pip install tensorflow_text >log.txt

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4")

In [ ]:
len(X),len(X_train),len(X_test)

(144293, 115434, 28859)

In [ ]:
X_train[:5]

117132    Some schools offer distance learning as an opt...
122438                                  have more free time
136832    Everybody has a different opinion since we are...
52562     The technology could tell what pecent of each ...
25077     Such as he knew it was an opprotunity of a lif...
Name: discourse_text, dtype: object

In [ ]:
y_train[:5]

array([[0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(7, activation='softmax', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model Run


In [ ]:
model.fit(X_train, y_train, 
          epochs = 5,
          batch_size = 5)

VER=3
model.save_weights(f'/content/drive/MyDrive/python/v{VER}.h5')
model.save(f'/content/drive/MyDrive/python/v{VER}model.h5')



Epoch 1/5
23087/23087 [==============================] - 2139s 92ms/step - loss: 1.4668 - accuracy: 0.5047
Epoch 2/5
23087/23087 [==============================] - 2144s 93ms/step - loss: 1.4277 - accuracy: 0.5158
Epoch 3/5
23087/23087 [==============================] - 2140s 93ms/step - loss: 1.4274 - accuracy: 0.5170
Epoch 4/5
23087/23087 [==============================] - 2139s 93ms/step - loss: 1.4299 - accuracy: 0.5171
Epoch 5/5
23087/23087 [==============================] - 2138s 93ms/step - loss: 1.4290 - accuracy: 0.5173


In [ ]:
model.evaluate(X_test, y_test)

902/902 [==============================] - 476s 526ms/step - loss: 1.6050 - accuracy: 0.5265


[1.605013132095337, 0.5264562368392944]

# Preprocessing B

In [ ]:
def type(x):
  if x=='Lead':
    return 0
  elif x=='Position':
    return 1
  elif x=='Evidence':
    return 2
  elif x=='Claim':
    return 3 
  elif x=='Concluding Statement':
    return 4
  elif x=='Counterclaim':
    return 5
  elif x=='Rebuttal':
    return 6

df['discourse_type_']=df.discourse_type.apply(type)
df

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,discourse_type_
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,0
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,1
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75,2
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,2
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...,3
...,...,...,...,...,...,...,...,...,...
144288,4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...,2
144289,4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...,2
144290,4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838,1
144291,4C471936CD75,1.618025e+12,4570.0,4922.0,The impact of asking people to help you make a...,Evidence,Evidence 4,839 840 841 842 843 844 845 846 847 848 849 85...,2


In [ ]:
from sklearn.model_selection import train_test_split
X=df['discourse_text']
y=df['discourse_type_']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)#,shuffle=False)

X_train[2],y_train[2]

('Some certain areas in the United States ban phones from class rooms just because of it. ',
 2)

In [ ]:
num_classes=7
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
y_train[2]

array([0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [ ]:
pip install tensorflow_text >log.txt

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_wwm_uncased_L-24_H-1024_A-16/4")

In [ ]:
len(X),len(X_train),len(X_test),len(X)-len(X_train)-len(X_test)

(144293, 115434, 28859, 0)

In [ ]:
X_train[:5].shape

(5,)

In [ ]:
X[122438]

'have more free time'

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(7, activation='softmax', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model run

In [ ]:
model.fit(X_train, y_train, 
          epochs = 5,
          batch_size = 5)

VER=4
model.save_weights(f'/content/drive/MyDrive/python/v{VER}.h5')
model.save(f'/content/drive/MyDrive/python/v{VER}model.h5')



Epoch 1/5
23087/23087 [==============================] - 2197s 94ms/step - loss: 1.4444 - accuracy: 0.5194
Epoch 2/5
23087/23087 [==============================] - 2171s 94ms/step - loss: 1.4081 - accuracy: 0.5290
Epoch 3/5
23087/23087 [==============================] - 2168s 94ms/step - loss: 1.4007 - accuracy: 0.5313
Epoch 4/5
23087/23087 [==============================] - 2166s 94ms/step - loss: 1.4052 - accuracy: 0.5301
Epoch 5/5
23087/23087 [==============================] - 2170s 94ms/step - loss: 1.4054 - accuracy: 0.5282


In [ ]:
model.evaluate(X_test, y_test)

902/902 [==============================] - 487s 539ms/step - loss: 1.1913 - accuracy: 0.5848


[1.1913479566574097, 0.5847742557525635]

# Best Model

## preprocessing

In [ ]:
pip install tensorflow_text >log.txt

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
def type(x):
  if x=='Lead':
    return 0
  elif x=='Position':
    return 1
  elif x=='Evidence':
    return 2
  elif x=='Claim':
    return 3 
  elif x=='Concluding Statement':
    return 4
  elif x=='Counterclaim':
    return 5
  elif x=='Rebuttal':
    return 6

df['discourse_type_']=df.discourse_type.apply(type)

from sklearn.model_selection import train_test_split
X=df['discourse_text']
y=df['discourse_type_']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)#,shuffle=False)

num_classes=7
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
X_train[10],y_train[10]

('Drivers should not be able to use phones while operating a vehicle.',
 array([0., 0., 1., 0., 0., 0., 0.], dtype=float32))

## Build model

In [ ]:
# Bert layers
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(7, activation='softmax', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

## Run and Save model

In [ ]:
model.fit(X_train, y_train, 
          epochs = 20,
          batch_size = 5)

VER=5
model.save_weights(f'/content/drive/MyDrive/python/v{VER}.h5')
model.save(f'/content/drive/MyDrive/python/v{VER}model.h5')



NameError: ignored

# Load model,weight and predict

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

VER=5
model = tf.keras.models.load_model(f'/content/drive/MyDrive/python/v{VER}model.h5',custom_objects={'KerasLayer':hub.KerasLayer})
model.load_weights(f'/content/drive/MyDrive/python/v{VER}.h5')


In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=1)
print('Accuracy: {:5.2f}%'.format(100 * acc))

902/902 [==============================] - 187s 203ms/step - loss: 1.0472 - accuracy: 0.6221
Accuracy: 62.21%


In [ ]:
test = ["Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving.",
        "They are some really bad consequences when stuff happens when it comes to a phone.",
        "Some certain areas in the United States ban phones from class rooms just because of it.",
        "When people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation. ",
        "Driving is one of the way how to get around. People always be on their phones while doing it. Which can cause serious Problems.",
        "That's why there's a thing that's called no texting while driving. That's a really important thing to remember. Some people still do it because they think It's stupid. No matter what they do they still have to obey it because that's the only way how did he save.",
        "Sometimes on the news there is either an accident or a suicide. It might involve someone not looking where they're going or tweet that someone sent. It either injury or death. If a mysterious number says I'm going to kill you and they know where you live but you don't know the person's contact,It makes you puzzled and make you start to freak out. Which can end up really badly. ",
        "Phones are fine to use and it's also the best way to come over help.",
        "If you go through a problem and you can't find help you ,always have a phone there with you. Even though phones are used almost every day as long as you're safe it would come into use if you get into trouble. Make sure you do not be like this phone while you're in the middle of driving.",
        "The news always updated when people do something stupid around that involves their phones. The safest way is the best way to stay safe."]

In [ ]:
prediction=model.predict(test)
prediction

array([[2.97298968e-01, 9.32165608e-03, 5.44627905e-01, 8.04875791e-02,
        4.52294908e-02, 6.09697821e-03, 1.69374738e-02],
       [2.69972403e-02, 1.00598246e-01, 1.54806077e-01, 6.44752502e-01,
        1.62515808e-02, 2.95412689e-02, 2.70530507e-02],
       [1.31171495e-01, 2.72398274e-02, 3.10550243e-01, 4.51477587e-01,
        2.97876471e-03, 7.28547499e-02, 3.72735341e-03],
       [8.82811189e-01, 4.40356525e-04, 1.01025321e-01, 1.86145003e-03,
        1.37559986e-02, 8.77804123e-05, 1.78843238e-05],
       [1.34409159e-01, 2.50552986e-02, 5.09489834e-01, 2.80762464e-01,
        2.97559015e-02, 1.47883799e-02, 5.73899969e-03],
       [7.43092820e-02, 1.04047479e-02, 6.87643945e-01, 6.17431141e-02,
        1.35074586e-01, 5.02372580e-03, 2.58005708e-02],
       [1.16645120e-01, 5.07463992e-04, 8.53271544e-01, 7.70212570e-03,
        1.93484426e-02, 1.64410356e-03, 8.81236629e-04],
       [2.50272602e-02, 2.14759305e-01, 1.52010486e-01, 5.67708552e-01,
        2.35981699e-02, 1

In [ ]:
predicted = np.argmax(prediction,axis=1)

###
#[4 5 3 3 0 3 3 0 3 1]
Actual=['Lead','Position','Evidence','Evidence','Claim','Evidence','Evidence','Claim','Evidence','Concluding Statement']
ans=[]
for i in Actual:
  ans.append(type(i))
###
#[0claim, 1concluding statement, 2counterclaim, 3evidence, 4lead, 5position, 6rebuttal]


print(f"Predicted: {predicted.tolist()}")
print(f'Actual:{ans}')
import numpy as np
np.array(predicted.tolist())-np.array(ans) #40%

Predicted: [2, 3, 3, 0, 2, 2, 2, 3, 2, 2]
Actual:[0, 1, 2, 2, 3, 2, 2, 3, 2, 4]


array([ 2,  2,  1, -2, -1,  0,  0,  0,  0, -2])

In [ ]:
def type(x):
  if x=='Lead':
    return 0
  elif x=='Position':
    return 1
  elif x=='Evidence':
    return 2
  elif x=='Claim':
    return 3 
  elif x=='Concluding Statement':
    return 4
  elif x=='Counterclaim':
    return 5
  elif x=='Rebuttal':
    return 6

In [ ]:
type('Lead')

0

In [ ]:
import random
sample_num=random.sample(range(len(df)),int(len(df)/100))

In [ ]:
test=df.iloc[sample_num][['discourse_text','discourse_type_']]
test.head(10)

,discourse_text,discourse_type_
102067,the Electoral College avoids the problem of el...,3
43009,"Here's my problem with the authors claims, an...",5
57606,"In the article ""Umasking the Face on Mars, in ...",3
114986,The whole point of school is to get you ready ...,0
129595,"They can stress much less, be able to manage t...",3
47859,i know all of this imformation because when im...,2
49235,In the passage Making Mona Lisa Smile they tal...,0
25237,all the countries you helped will remember you...,2
81253,"So in concluson, driverless cars are unsafe fo...",4
2191,Most people don't see the dangers but peoples ...,2


In [ ]:
X_sample=df.iloc[sample_num][['discourse_text','discourse_type_']]['discourse_text']
X_sample[:10]

102067    the Electoral College avoids the problem of el...
43009      Here's my problem with the authors claims, an...
57606     In the article "Umasking the Face on Mars, in ...
114986    The whole point of school is to get you ready ...
129595    They can stress much less, be able to manage t...
47859     i know all of this imformation because when im...
49235     In the passage Making Mona Lisa Smile they tal...
25237     all the countries you helped will remember you...
81253     So in concluson, driverless cars are unsafe fo...
2191      Most people don't see the dangers but peoples ...
Name: discourse_text, dtype: object

In [ ]:
prediction=model.predict(X_sample)
predicted = np.argmax(prediction,axis=1)
test['prediction']=predicted
test.head(10)

,discourse_text,discourse_type_,prediction
102067,the Electoral College avoids the problem of el...,3,3
43009,"Here's my problem with the authors claims, an...",5,2
57606,"In the article ""Umasking the Face on Mars, in ...",3,2
114986,The whole point of school is to get you ready ...,0,2
129595,"They can stress much less, be able to manage t...",3,3
47859,i know all of this imformation because when im...,2,2
49235,In the passage Making Mona Lisa Smile they tal...,0,2
25237,all the countries you helped will remember you...,2,4
81253,"So in concluson, driverless cars are unsafe fo...",4,4
2191,Most people don't see the dangers but peoples ...,2,2


In [ ]:
true=0
for i in sample_num:
  if test.loc[i]['discourse_type_']==test.loc[i]['prediction']:
    true+=1

print(f'Accuracy: {round(true/len(test),4)*100}%')

Accuracy: 61.58%


In [ ]:
import random
#random sampling
sample_num=random.sample(range(len(df)),int(len(df)/100))
test=df.iloc[sample_num][['discourse_text','discourse_type_']]

#define sample for Validation
X_sample=df.iloc[sample_num][['discourse_text','discourse_type_']]['discourse_text']

#prediction
prediction=model.predict(X_sample)
predicted = np.argmax(prediction,axis=1)
test['prediction']=predicted

#calculate accuracy
true=0
for i in sample_num:
  if test.loc[i]['discourse_type_']==test.loc[i]['prediction']:
    true+=1

print(f'Accuracy: {round(true/len(test),4)*100}%')

Accuracy: 61.23%


# Demo sample essay

In [ ]:
p4="""The lack of a new generation is also to be blamed for the vanquishing of independent stationery shops when Mary, an independent stationery shop owner to be returned soon, asks her son to operate the shop for her, her son said. I would rather work in a large book store. It’s more comfortable and has a higher salary. Under the booming economy, the new generation of teenagers is desperate for a higher salary for their lavish expenditure. In stark contrast, the independent stationery stores are earning fewer year by year, so are the owners and employees running the shop. Knowing that there is little to earn, it is no stranger that teenagers are reluctant to join the small stationery sellers. Instead, they would rather become the employers of large book stores, or run electronic shops of their own. As decades passed, owners of these independent book stores became too old to run the shops. Despite the efforts they paid, it is futile for them to seek a successor. Feeling tired, sad, and frustrated, the shop owners had no choice but to shut down their stores permanently."""
p5="""It is the skyrocketing rent in Hong Kong, however, that should bear the greatest brunt of the saddening trend. Peter, who closed down a few months ago, claimed that he would have earned enough to make a living, had the rent been a little lower. The price of housing flats has been drastically soaring since the end of SARS, and the shop rent has been following suit. While the independent stationery stores have been sparing no paths to make ends meet, the profit they made is far from enough to chase the increase in Hong Kong’s rent. Owing to the high rent introduced by the flat owners and general economic conditions, it is blatantly expensive that the independent stationery shops can hardly make it through, although they tried to survive with the connection and network it had built over decades."""

In [ ]:
para4=[]
para5=[]
for i in p4.split("."):
  if len(i)>1:
    para4.append(i.strip()+'.')
for i in p5.split("."):
  if len(i)>1:
    para5.append(i.strip()+'.')

In [ ]:
para4[0]

'The lack of a new generation is also to be blamed for the vanquishing of independent stationery shops when Mary, an independent stationery shop owner to be returned soon, asks her son to operate the shop for her, her son said.'

In [ ]:
dict={ 0:'Lead',1:'Position',2:'Evidence',3:'Claim',4:'Concluding Statement',5:'Counterclaim',6:'Rebuttal'}
prediction=model.predict(para4)
predicted = np.argmax(prediction,axis=1)


In [ ]:
P4=pd.DataFrame(para4)
P4['type']=[dict.get(i) for i in predicted.tolist()]
P4

,0,type
0,The lack of a new generation is also to be bla...,Evidence
1,I would rather work in a large book store.,Position
2,It’s more comfortable and has a higher salary.,Claim
3,"Under the booming economy, the new generation ...",Claim
4,"In stark contrast, the independent stationery ...",Claim
5,"Knowing that there is little to earn, it is no...",Lead
6,"Instead, they would rather become the employer...",Evidence
7,"As decades passed, owners of these independent...",Evidence
8,"Despite the efforts they paid, it is futile fo...",Claim
9,"Feeling tired, sad, and frustrated, the shop o...",Lead


In [ ]:
prediction=model.predict(para5)
predicted = np.argmax(prediction,axis=1)
P5=pd.DataFrame(para5)
P5['type']=[dict.get(i) for i in predicted.tolist()]
P5

,0,type
0,"It is the skyrocketing rent in Hong Kong, howe...",Position
1,"Peter, who closed down a few months ago, claim...",Evidence
2,The price of housing flats has been drasticall...,Claim
3,While the independent stationery stores have b...,Claim
4,Owing to the high rent introduced by the flat ...,Evidence


In [ ]:
P5["type"][1],P5[0][1]

('Evidence',
 'Peter, who closed down a few months ago, claimed that he would have earned enough to make a living, had the rent been a little lower.')

In [ ]:
P5["type"][2],P5[0][2]

('Claim',
 'The price of housing flats has been drastically soaring since the end of SARS, and the shop rent has been following suit.')

In [ ]:
P4["type"][2],P4[0][2]

('Claim', 'It’s more comfortable and has a higher salary.')